can you hear me 

This is a crawler to get the cover picture of movies in our data from douban website.

In [1]:
import pandas as pd 
import numpy as np 
df1=pd.read_csv('../data/tmdb-movie-metadata/tmdb_5000_credits.csv')
df2=pd.read_csv('../data/tmdb-movie-metadata/tmdb_5000_movies.csv')

In [3]:
df1.columns = ['id','tittle','cast','crew']
df2= df2.merge(df1,on='id')

this is our dataset.
it's like.

In [101]:
df2.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,spoken_languages,status,tagline,title,vote_average,vote_count,tittle,cast,crew,cover
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",https://img1.doubanio.com/view/photo/s_ratio_p...
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...",https://img9.doubanio.com/view/photo/s_ratio_p...
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...",https://img9.doubanio.com/view/photo/s_ratio_p...
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...",https://img9.doubanio.com/view/photo/s_ratio_p...
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...",https://img1.doubanio.com/view/photo/s_ratio_p...


In [31]:
import requests
from bs4 import BeautifulSoup
import urllib.request

we can get the name of movies, and search them in douban website.

the function is here

In [25]:
def getsoup(url):
    r = requests.get(url)
    r.encoding = 'utf-8'  # 
    soup = BeautifulSoup(r.text, 'lxml')
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:23.0) Gecko/20100101 Firefox/23.0'}
    req = urllib.request.Request(url,headers=headers)  
    webpage = urllib.request.urlopen(req)  
    html = webpage.read()
    soup = BeautifulSoup(html, 'html.parser')
    return soup

def getpic1(result):
    img = result.findAll("img")[0].attrs["src"]
    imgurl = img
    file_suffix = os.path.splitext(img)[1]
    file_name = img.split('/')[-1].split('.jpg')[0]
    filename = '{}{}{}{}'.format(file_path,os.sep,file_name,file_suffix)
    return img,imgurl,file_suffix,file_name,filename

for example

for a movie called 'My Date with Drew',
we search it in the website,
then we can get a picutre of its cover

so we can get the url of the movie,
then get the url of the picture

In [21]:
tittle = 'My Date with Drew'
movie_url = 'https://www.douban.com/search?q=%s'%(tittle.replace(' ','+'))

and the results are here

In [47]:
soup = getsoup(movie_url)
results = soup.find_all(attrs={"class": "result-list"})
movie_rts = results[0].findAll(attrs={'class':"result"})

it will get all the movies searched,
but we just need the first one,
becuase it's the most close to our search

then we get the result's img attribution, it's the cover,
like this

after that, we can do the same thing for all movies in the dataset

In [59]:
print(movie_rts[0].img.attrs['src'])

https://img3.doubanio.com/view/subject/s/public/s2566670.jpg


here is a trivial thing,
there maybe some special character in the movie's name,
like, ":" ,"'", and something else,
so we need to clear them to get the url,
the function is below
(but there maybe some other situations, the function needs improving)

In [73]:
def rename_title(tittle):
    tittle = tittle.replace(' ','+')
    tittle = tittle.replace(':','%3A')
    tittle = tittle.replace('\'','%27')
    return tittle

then we repeat the crawler for the dataset
there are 4803 movies

In [104]:
len(df3)

4803

however, lots of movies get failure

In [86]:
failure_url = []
for i in range(len(df2)):
    tittle = df2['original_title'][i]
    tittle = rename_title(tittle)
    movie_url = 'https://www.douban.com/search?cat=1002&q=%s'%(tittle)
    try:
        soup = getsoup(movie_url)
        results = soup.find_all(attrs={"class": "result-list"})
        movie_rts = results[0].findAll(attrs={'class':"result"})
        df3['cover'][i] = movie_rts[0].img.attrs['src']
    except:
        failure_url.append(movie_url)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


https://www.douban.com/search?cat=1002&q=WALL·E
https://www.douban.com/search?cat=1002&q=シン・ゴジラ
https://www.douban.com/search?cat=1002&q=Astérix+aux+Jeux+Olympiques
https://www.douban.com/search?cat=1002&q=金陵十三釵
https://www.douban.com/search?cat=1002&q=Déjà+Vu
https://www.douban.com/search?cat=1002&q=Évolution
https://www.douban.com/search?cat=1002&q=Deux+frères
https://www.douban.com/search?cat=1002&q=Les+Misérables
https://www.douban.com/search?cat=1002&q=天將雄師
https://www.douban.com/search?cat=1002&q=西游记之孙悟空三打白骨精
https://www.douban.com/search?cat=1002&q=Alien³
https://www.douban.com/search?cat=1002&q=Æon+Flux
https://www.douban.com/search?cat=1002&q=Un+long+dimanche+de+fiançailles
https://www.douban.com/search?cat=1002&q=Солярис
https://www.douban.com/search?cat=1002&q=Last+Holiday
https://www.douban.com/search?cat=1002&q=The+River+Wild
https://www.douban.com/search?cat=1002&q=The+Indian+in+the+Cupboard
https://www.douban.com/search?cat=1002&q=Savages
https://www.douban.com/search?ca

https://www.douban.com/search?cat=1002&q=The+Doors
https://www.douban.com/search?cat=1002&q=Sex+Tape
https://www.douban.com/search?cat=1002&q=Hanging+Up
https://www.douban.com/search?cat=1002&q=Final+Destination+5
https://www.douban.com/search?cat=1002&q=Le+fabuleux+destin+d%27Amélie+Poulain
https://www.douban.com/search?cat=1002&q=投名狀
https://www.douban.com/search?cat=1002&q=Un+monstre+à+Paris
https://www.douban.com/search?cat=1002&q=一代宗師
https://www.douban.com/search?cat=1002&q=A+Man+Apart
https://www.douban.com/search?cat=1002&q=Aloha
https://www.douban.com/search?cat=1002&q=Ghosts+of+Mississippi
https://www.douban.com/search?cat=1002&q=Snow+Falling+on+Cedars
https://www.douban.com/search?cat=1002&q=The+Rite
https://www.douban.com/search?cat=1002&q=Gattaca
https://www.douban.com/search?cat=1002&q=Isn%27t+She+Great
https://www.douban.com/search?cat=1002&q=Space+Chimps
https://www.douban.com/search?cat=1002&q=Head+of+State
https://www.douban.com/search?cat=1002&q=The+Hangover
https://

https://www.douban.com/search?cat=1002&q=Se7en
https://www.douban.com/search?cat=1002&q=District+9
https://www.douban.com/search?cat=1002&q=The+SpongeBob+SquarePants+Movie
https://www.douban.com/search?cat=1002&q=Mystic+River
https://www.douban.com/search?cat=1002&q=Analyze+This
https://www.douban.com/search?cat=1002&q=The+Notebook
https://www.douban.com/search?cat=1002&q=27+Dresses
https://www.douban.com/search?cat=1002&q=Hannah+Montana%3A+The+Movie
https://www.douban.com/search?cat=1002&q=Rugrats+in+Paris%3A+The+Movie
https://www.douban.com/search?cat=1002&q=Legends+of+the+Fall
https://www.douban.com/search?cat=1002&q=Up+in+the+Air
https://www.douban.com/search?cat=1002&q=Babe
https://www.douban.com/search?cat=1002&q=Hope+Springs
https://www.douban.com/search?cat=1002&q=Forgetting+Sarah+Marshall
https://www.douban.com/search?cat=1002&q=Four+Brothers
https://www.douban.com/search?cat=1002&q=Baby+Mama
https://www.douban.com/search?cat=1002&q=Hope+Floats
https://www.douban.com/search?ca

https://www.douban.com/search?cat=1002&q=Miracle
https://www.douban.com/search?cat=1002&q=Dawn+of+the+Dead
https://www.douban.com/search?cat=1002&q=キャプテンハーロック
https://www.douban.com/search?cat=1002&q=The+Royal+Tenenbaums
https://www.douban.com/search?cat=1002&q=Last+Vegas
https://www.douban.com/search?cat=1002&q=For+Your+Eyes+Only
https://www.douban.com/search?cat=1002&q=Serendipity
https://www.douban.com/search?cat=1002&q=Timecop
https://www.douban.com/search?cat=1002&q=Zoolander
https://www.douban.com/search?cat=1002&q=Safe+Haven
https://www.douban.com/search?cat=1002&q=Hocus+Pocus
https://www.douban.com/search?cat=1002&q=No+Reservations
https://www.douban.com/search?cat=1002&q=Dracula+2000
https://www.douban.com/search?cat=1002&q=Alexander+and+the+Terrible,+Horrible,+No+Good,+Very+Bad+Day
https://www.douban.com/search?cat=1002&q=Pride+&+Prejudice
https://www.douban.com/search?cat=1002&q=Blade+Runner
https://www.douban.com/search?cat=1002&q=Rob+Roy
https://www.douban.com/search?cat=1

...

https://www.douban.com/search?cat=1002&q=Spellbound
https://www.douban.com/search?cat=1002&q=Exotica
https://www.douban.com/search?cat=1002&q=Buffalo+%2766
https://www.douban.com/search?cat=1002&q=Insidious
https://www.douban.com/search?cat=1002&q=Repo+Man
https://www.douban.com/search?cat=1002&q=Nueve+Reinas
https://www.douban.com/search?cat=1002&q=The+Gatekeepers
https://www.douban.com/search?cat=1002&q=The+Ballad+of+Jack+and+Rose
https://www.douban.com/search?cat=1002&q=The+To+Do+List
https://www.douban.com/search?cat=1002&q=Killing+Zoe
https://www.douban.com/search?cat=1002&q=The+Believer
https://www.douban.com/search?cat=1002&q=Snow+Angels
https://www.douban.com/search?cat=1002&q=Unsullied
https://www.douban.com/search?cat=1002&q=Session+9
https://www.douban.com/search?cat=1002&q=I+Want+Someone+to+Eat+Cheese+With
https://www.douban.com/search?cat=1002&q=Mooz-lum
https://www.douban.com/search?cat=1002&q=Hatchet
https://www.douban.com/search?cat=1002&q=Modern+Times
https://www.douba

https://www.douban.com/search?cat=1002&q=20+Feet+from+Stardom
https://www.douban.com/search?cat=1002&q=Two+Girls+and+a+Guy
https://www.douban.com/search?cat=1002&q=Walking+and+Talking
https://www.douban.com/search?cat=1002&q=Who+Killed+the+Electric+Car?
https://www.douban.com/search?cat=1002&q=The+Broken+Hearts+Club%3A+A+Romantic+Comedy
https://www.douban.com/search?cat=1002&q=Bubba+Ho-tep
https://www.douban.com/search?cat=1002&q=Slam
https://www.douban.com/search?cat=1002&q=Brigham+City
https://www.douban.com/search?cat=1002&q=Fiza
https://www.douban.com/search?cat=1002&q=Orgazmo
https://www.douban.com/search?cat=1002&q=All+the+Real+Girls
https://www.douban.com/search?cat=1002&q=Dream+with+the+Fishes
https://www.douban.com/search?cat=1002&q=Blue+Car
https://www.douban.com/search?cat=1002&q=Palo+Alto
https://www.douban.com/search?cat=1002&q=Ajami
https://www.douban.com/search?cat=1002&q=Wristcutters%3A+A+Love+Story
https://www.douban.com/search?cat=1002&q=I+Origins
https://www.douban.c

https://www.douban.com/search?cat=1002&q=Médecin+de+campagne
https://www.douban.com/search?cat=1002&q=The+Maid%27s+Room
https://www.douban.com/search?cat=1002&q=A+Hard+Day%27s+Night
https://www.douban.com/search?cat=1002&q=The+Harvest+(La+Cosecha)
https://www.douban.com/search?cat=1002&q=Love+Letters
https://www.douban.com/search?cat=1002&q=Julija+in+Alfa+Romeo
https://www.douban.com/search?cat=1002&q=Fireproof
https://www.douban.com/search?cat=1002&q=Faith+Connections
https://www.douban.com/search?cat=1002&q=Benji
https://www.douban.com/search?cat=1002&q=Open+Water
https://www.douban.com/search?cat=1002&q=High+Road
https://www.douban.com/search?cat=1002&q=Kingdom+of+the+Spiders
https://www.douban.com/search?cat=1002&q=Mad+Hot+Ballroom
https://www.douban.com/search?cat=1002&q=The+Station+Agent
https://www.douban.com/search?cat=1002&q=To+Save+A+Life
https://www.douban.com/search?cat=1002&q=Wordplay
https://www.douban.com/search?cat=1002&q=Beyond+the+Mat
https://www.douban.com/search?cat

https://www.douban.com/search?cat=1002&q=Sugar+Town
https://www.douban.com/search?cat=1002&q=The+Motel
https://www.douban.com/search?cat=1002&q=The+Canyons
https://www.douban.com/search?cat=1002&q=On+the+Outs
https://www.douban.com/search?cat=1002&q=Shotgun+Stories
https://www.douban.com/search?cat=1002&q=Exam
https://www.douban.com/search?cat=1002&q=The+Sticky+Fingers+of+Time
https://www.douban.com/search?cat=1002&q=Sunday+School+Musical
https://www.douban.com/search?cat=1002&q=Rust
https://www.douban.com/search?cat=1002&q=Ink
https://www.douban.com/search?cat=1002&q=The+Christmas+Bunny
https://www.douban.com/search?cat=1002&q=Butterfly
https://www.douban.com/search?cat=1002&q=UnDivided
https://www.douban.com/search?cat=1002&q=The+Frozen
https://www.douban.com/search?cat=1002&q=Horse+Camp
https://www.douban.com/search?cat=1002&q=Give+Me+Shelter
https://www.douban.com/search?cat=1002&q=The+Big+Parade
https://www.douban.com/search?cat=1002&q=Little+Big+Top
https://www.douban.com/search?

https://www.douban.com/search?cat=1002&q=Pink+Flamingos
https://www.douban.com/search?cat=1002&q=Clean
https://www.douban.com/search?cat=1002&q=دایره
https://www.douban.com/search?cat=1002&q=Tin+Can+Man
https://www.douban.com/search?cat=1002&q=キュア
https://www.douban.com/search?cat=1002&q=On+The+Downlow
https://www.douban.com/search?cat=1002&q=Sanctuary%3A+Quite+a+Conundrum
https://www.douban.com/search?cat=1002&q=Bang
https://www.douban.com/search?cat=1002&q=Primer
https://www.douban.com/search?cat=1002&q=Cavite
https://www.douban.com/search?cat=1002&q=El+Mariachi
https://www.douban.com/search?cat=1002&q=Newlyweds
https://www.douban.com/search?cat=1002&q=Signed,+Sealed,+Delivered
https://www.douban.com/search?cat=1002&q=Shanghai+Calling
https://www.douban.com/search?cat=1002&q=My+Date+with+Drew


we see the failure url, although we can open the url successfuly by hand,
but the "soup" seems can't get it right,
for example, the url which concludes japanese character,
https://www.douban.com/search?cat=1002&q=%E3%82%B7%E3%83%B3%E3%83%BB%E3%82%B4%E3%82%B8%E3%83%A9,
its url should be this,
but we can't get the character switch automately now...

fortunately, there are 1488 movies got success, and I stored their cover url in dataframe

In [99]:
success = df3[['original_title','cover']]
success = success[success['cover']!='']
success = success.reset_index(drop=True)
success

,original_title,cover
0,Avatar,https://img1.doubanio.com/view/photo/s_ratio_p...
1,Pirates of the Caribbean: At World's End,https://img9.doubanio.com/view/photo/s_ratio_p...
2,Spectre,https://img9.doubanio.com/view/photo/s_ratio_p...
3,The Dark Knight Rises,https://img9.doubanio.com/view/photo/s_ratio_p...
4,John Carter,https://img1.doubanio.com/view/photo/s_ratio_p...
...,...,...
1483,The Last Sin Eater,https://img3.doubanio.com/view/photo/s_ratio_p...
1484,Mi America,https://img1.doubanio.com/view/photo/s_ratio_p...
1485,Silent House,https://img9.doubanio.com/view/photo/s_ratio_p...
1486,Full Frontal,https://img1.doubanio.com/view/photo/s_ratio_p...


for example, we can open the url and get jpg,

In [100]:
for i in range(len(success)):
    print(success.cover[i])

https://img1.doubanio.com/view/photo/s_ratio_poster/public/p2180085848.jpg
https://img9.doubanio.com/view/photo/s_ratio_poster/public/p1076818175.jpg
https://img9.doubanio.com/view/photo/s_ratio_poster/public/p2276234635.jpg
https://img9.doubanio.com/view/photo/s_ratio_poster/public/p1706428744.jpg
https://img1.doubanio.com/view/photo/s_ratio_poster/public/p1560217027.jpg
https://img1.doubanio.com/view/photo/s_ratio_poster/public/p1344981868.jpg
https://img1.doubanio.com/view/photo/s_ratio_poster/public/p707509468.jpg
https://img3.doubanio.com/view/photo/s_ratio_poster/public/p2237747953.jpg
https://img9.doubanio.com/view/photo/s_ratio_poster/public/p1108070214.jpg
https://img9.doubanio.com/view/photo/s_ratio_poster/public/p2316834186.jpg
https://img3.doubanio.com/view/photo/s_ratio_poster/public/p457079860.jpg
https://img3.doubanio.com/view/photo/s_ratio_poster/public/p976999823.jpg
https://img3.doubanio.com/view/photo/s_ratio_poster/public/p1076822803.jpg
https://img3.doubanio.com/vi

then it can be used in the development of ZHONG Hui!
This is what I do... Thank you!
Do you have questions...